In [1]:
import pandas as pd
from datetime import datetime, date
from pathlib import Path

# === File paths (adjust if needed) ===
CURRENT_XLSX = Path("C:\\Users\\User\\OneDrive\\Momentum\\Scripts\\Signal_Archive\\Current.xlsx")
TODAYS_CSV   = Path("G:\\My Drive\\Deep_Learn_Analysis\\Daily\\todays_entry_review.csv")

# === Columns to compare/update ===
PARAM_COLS = ["n_components", "poly_degree", "std_multiplier", "train_window"]

# === Extra fields to update when a change occurs (not part of comparison) ===
TODAY_TO_CURRENT_EXTRA = {
    "Strategy Run Rate": "Return",
    "# of trades": "Transactions",
    "Min_StopLoss%": "Stop Loss",
}

# --- Load data ---
current_df = pd.read_excel(CURRENT_XLSX)          # uses first sheet by default
today_df   = pd.read_csv(TODAYS_CSV)

# --- Normalize column names minimally (handles common variant n_components) ---
rename_map = {"n_components": "n_components"}  # add more variants if you have them
current_df = current_df.rename(columns=rename_map)
today_df   = today_df.rename(columns=rename_map)

# --- Basic validation ---
required_in_current = ["Ticker"] + PARAM_COLS
required_in_today   = ["Ticker"] + PARAM_COLS

missing_current = [c for c in required_in_current if c not in current_df.columns]
missing_today   = [c for c in required_in_today   if c not in today_df.columns]

if missing_current:
    raise KeyError(f"Missing columns in Current.xlsx: {missing_current}")
if missing_today:
    raise KeyError(f"Missing columns in todays_entry_review.csv: {missing_today}")

# Ensure extra target columns exist in Current.xlsx (create if absent)
for _, dst_col in TODAY_TO_CURRENT_EXTRA.items():
    if dst_col not in current_df.columns:
        current_df[dst_col] = pd.NA

# Ensure Date column exists
if "Date" not in current_df.columns:
    current_df["Date"] = pd.NaT

# --- Create a case-insensitive key for matching tickers (does not alter original Ticker) ---
current_df["_ticker_key"] = current_df["Ticker"].astype(str).str.strip().str.upper()
today_df["_ticker_key"]   = today_df["Ticker"].astype(str).str.strip().str.upper()

# Keep only the first occurrence per ticker in today's file (if duplicates exist)
today_dedup = today_df.drop_duplicates(subset=["_ticker_key"], keep="first").copy()

# --- Ensure numeric comparison for parameter columns ---
for col in PARAM_COLS:
    current_df[col]   = pd.to_numeric(current_df[col], errors="coerce")
    today_dedup[col]  = pd.to_numeric(today_dedup[col], errors="coerce")

# Coerce extra-source columns to numeric where appropriate
for src_col in TODAY_TO_CURRENT_EXTRA.keys():
    if src_col in today_dedup.columns:
        today_dedup[src_col] = pd.to_numeric(today_dedup[src_col], errors="coerce")

# --- Index today's data by ticker key for fast lookups ---
today_idx = today_dedup.set_index("_ticker_key")

# --- Iterate and update where different ---
changed_rows = []
missing_in_current = []   # tickers in today's CSV that aren't in current
seen_in_current_keys = set(current_df["_ticker_key"].unique())

# Track today-only tickers for info (not appended)
today_only_keys = set(today_idx.index) - seen_in_current_keys
if today_only_keys:
    missing_in_current = list(today_only_keys)

# Helper for equality that treats NaNs as equal only if both are NaN
def values_different(a, b) -> bool:
    if pd.isna(a) and pd.isna(b):
        return False
    return a != b

today_str = date.today().isoformat()  # e.g., '2025-08-11'

for i, row in current_df.iterrows():
    k = row["_ticker_key"]
    if k in today_idx.index:
        changed_cols = []
        # Compare only the parameter columns
        for col in PARAM_COLS:
            old_val = row[col]
            new_val = today_idx.at[k, col]
            if values_different(old_val, new_val):
                current_df.at[i, col] = new_val
                changed_cols.append((col, old_val, new_val))

        if changed_cols:
            # Update the extra fields (not part of comparison)
            for src_col, dst_col in TODAY_TO_CURRENT_EXTRA.items():
                if src_col in today_idx.columns:
                    current_df.at[i, dst_col] = today_idx.at[k, src_col]

            # Update the Date column to today's date
            current_df.at[i, "Date"] = today_str

            changed_rows.append({
                "Ticker": row["Ticker"],
                "Changed_Parameter_Columns": [c[0] for c in changed_cols],
                "Details": changed_cols,
                "Also_Updated": list(TODAY_TO_CURRENT_EXTRA.values()) + ["Date"],
            })

# --- Save results: make a timestamped backup, then overwrite Current.xlsx ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_path = CURRENT_XLSX.with_name(f"{CURRENT_XLSX.stem}_BACKUP_{timestamp}{CURRENT_XLSX.suffix}")

with pd.ExcelWriter(backup_path, engine="openpyxl") as wb:
    current_df.drop(columns=["_ticker_key"]).to_excel(wb, index=False)

with pd.ExcelWriter(CURRENT_XLSX, engine="openpyxl") as wb:
    current_df.drop(columns=["_ticker_key"]).to_excel(wb, index=False)

# --- Report ---
if changed_rows:
    print("✅ Updated the following tickers in Current.xlsx (and created a backup):")
    for item in changed_rows:
        ticker = item["Ticker"]
        cols = ", ".join(item["Changed_Parameter_Columns"])
        print(f"  - {ticker}: parameter changes -> {cols}; extras updated -> Return, Transactions, Stop Loss, Date")
else:
    print("ℹ️ No parameter differences found. Current.xlsx was left unchanged (but a backup was still created).")

if missing_in_current:
    print("\nNote: These tickers exist in todays_entry_review.csv but not in Current.xlsx (not appended):")
    for k in sorted(missing_in_current):
        # Show original-case ticker from today's file for readability
        orig = today_dedup.loc[today_dedup["_ticker_key"] == k, "Ticker"].iloc[0]
        print(f"  - {orig}")

# Optional: also write a CSV of changed tickers for your records
if changed_rows:
    flat_rows = []
    for item in changed_rows:
        ticker = item["Ticker"]
        for col, old_val, new_val in item["Details"]:
            flat_rows.append({
                "Ticker": ticker,
                "Column": col,
                "Old_Value": old_val,
                "New_Value": new_val
            })
    pd.DataFrame(flat_rows).to_csv("replaced_tickers_detail.csv", index=False)
    print("\n📄 Wrote detailed parameter changes to replaced_tickers_detail.csv")

✅ Updated the following tickers in Current.xlsx (and created a backup):
  - VCX.AX: parameter changes -> poly_degree, train_window; extras updated -> Return, Transactions, Stop Loss, Date

Note: These tickers exist in todays_entry_review.csv but not in Current.xlsx (not appended):
  - IT
  - TPG.AX

📄 Wrote detailed parameter changes to replaced_tickers_detail.csv
